# Cifrario di Hill
## Implementiamo il cifrario di Hill in python
Per prima cosa importiamo numpy che gestisce già le moltiplicazioni tra matrici e tra vettori.

In [37]:
import numpy as np
import scipy as sc

Definiamo quindi una funzione che si appoggi alla moltiplicazione matriciale di numpy ma la renda modulo 26.

In [3]:
def mulmod(A,B):
    C=np.dot(A,B)
    d=C.shape
    
    for i in range(d[0]):
        for j in range(d[1]):
            C[i,j]=C[i,j]%26
    return C

Definiamo una lista che colleghi le lettere ai valori numerici.

In [4]:
s="abcdefghijklmnopqrstuvwxyz"
alfabeto=list(s)
r=range(26)
D=dict(zip(alfabeto,r))
DD=dict(zip(r,alfabeto))

Estraiamo il testo da un file txt.

In [5]:
file=open("Plain.txt","r")
PT=[]
for line in file:
    PT.append(line)

file.close()

Ripuliamo il messaggio.

In [6]:
PT="".join(PT)
PT=PT.split(" ")
PT="".join(PT)
PT=PT.split("\n")
PT="".join(PT)
PT=PT.split("'")
PT="".join(PT)
PT=PT.lower()


Trasformiamo il messaggio in un  vettore numerico e lo dividiamo in blocchi di dimensione fissata m con un reshape.

In [7]:
m=2
P=list(PT)
P=[ D[P[i]] for i in range(len(P))]
n=len(P)
print(n)
P=P+[0]*(m-n%m)
#print(P)
P=np.array(P)
print(P)
n=len(P)
righe=int(n/m)


P=np.reshape(P,(righe,m))
P

1477
[13 14 13 ... 11  4  0]


array([[13, 14],
       [13, 15],
       [20, 14],
       ...,
       [20, 21],
       [14, 11],
       [ 4,  0]])

In [8]:
P[295,:]

array([ 0, 13])

Definiamo una chiave K

In [9]:
K=np.array([[1,2],[2,3]])

Cifriamo tramite la moltiplicazione matriciale modulo 26 e otteniamo il testo cifrato riutilizzando la corridpondenza lettere numeri.

In [10]:
C=mulmod(P,K)
CT=np.reshape(C,(1,n))
CT=CT[0]
CT=[DD[i] for i in CT]
CT="".join(CT)

In [11]:
CT

'pqrtwemwmmbcfrmhsqnauumhridguanmexqmwbmuubizmyhuriscskbhykkcmwopgsmyqlkjbyqzqclszuqygtsisfetoirimybcepoixckjsvkcuaeuukexxdgackmmyeqvlwzenaxqvopadgtswbidopiidclseoanbyxgxbsdqyfcanvciinawqokkjqciiagwbevapqmwbkromuopcubwhblyeiqooanoybueubinalwuaxbanvcmhwywqqpcsocxglwzuuanmzuepanfwzepqegcezfochdwhblsiykcwvmtyoirtkcuadgbyoxmhukzuooexdczfsimmkrevblocctwyrigmdygsxqzutymwmmugwobyawqlrimjoiriwhnmriephgzsgaexcsanmjmhqzokiqooidlorgktexybmwidqcenubeoeitjtyskrimwopukgsxqzutysskjeawobyroqasikqtefcauwkegpaskbhaneajgopfwdghdevkzkjiossopyrmhuviopapqxcevutanpqvmkievsikjbyqrsvwyxdyroplsteeoapnmqzokcsjbanauuohgwqopyesvrilwxqhdcsqyxpopfwjkceobiqpqlszftyjedcbkmfsiyeuvanbylwkdgaoxidocfwteizcsokbycmmmcqgzybkjtmqzsdahsicevcexhatjahyumhexyebycmqsnkuotjyeiqooteepanjkaudyxgxbegpafcanwkfcanoydmzuexscapqshbopjeooxbulgjizjkmyjkwkcnsipacekiidmhocubpecmulmfmunwpsmhcseabhevvceuuklszftymsbucnmsanssyhmhmheaeuukkgqnokeidpqypecnaahdwbkcblidwkcwtimydyxqvopaiiykeadlidscrabuloscdygyjytwcaanlfyeizqyeulkokyhmhmheauanmkgqnkrcai

## Inversione e Decifratura
Costruiamo una funzione che calocli, se possibile, la matrice inversa di K e usiamola per decifrare il testo.

In [48]:
def invertibile(A):
    det=round(sc.linalg.det(A))
    return not (det%13==0 or det%2==0)

    

In [58]:
def inversa(A):
    det=sc.linalg.det(A)%26
    invdet=0
    if det%26==2 or det%26==13 or det%26==0:
        return 0
    while det*invdet%26!=1:
        invdet+=1
    
    print(invdet)
    inv=[]
    d=A.shape
    
    for j in range(d[1]):
        riga=[]
        for i in range(d[0]):
            righe=list(range(d[0]))
            righe.remove(i)
            colonne=list(range(d[1]))
            colonne.remove(j)
            parita=(-1)**(((i+1)+(j+1)))
            rid=A[righe,colonne]
            #print(type(rid[0]))
            if 1 in rid.shape:
                detrid=rid[0]
            else:
                detrid=np.linalg.det(rid)%26
            #print(type(parita), type(invdet), type(detrid))
            riga.append((parita*detrid*invdet)%26)
        inv.append(riga)
    return np.array(inv)

In [50]:
Inv=inversa(K)
Inv

array([[23,  2],
       [ 2, 25]])

In [51]:
np.dot(K,Inv)%26

array([[1, 0],
       [0, 1]], dtype=int32)

In [52]:
PP=np.dot(C,Inv)%26
PPP=np.reshape(PP,(1,n))[0]
PPP="".join([DD[i] for i in PPP])
PPP

'nonpuoicombattereunaguerradasoloilcuoreeunarmaturacisalvamasiconsumaavoltechiedereaiutocifapauramabastaunsolopassocomeilprimouomosullalunaperchedafuorinonsivedequantevoltehaipiantosinascesoliesimuorenelcuorediqualcunaltrosiamoangeliconunalasoltantoeriusciremoavolaresolorestandolunoaccantoallaltrocammineroaunpassodateefermeremoilventocomedentroagliuraganisupereroicomeioeteseavraipauraallorastringimilemaniperchesiamoinvincibilivicinieovunqueandrosaraiconmesupereroisoloioeteduegoccedipioggiachesalvanoilmondodallenuvolecisonoferitechenonsenevannonemmenocoltempopiuprofondediquellochesembranoguarisconosopralapellemainfondoticambianodentrohoversatocositantelacrimefinoadodiaremestessomaognivoltachehotoccatoilfondotucerilostessoohquandosiamodistantiognivoltachepiangipiangepureilcieloohnonhomoltodadartimatigiurochecammineroaunpassodateefermeremoilventocomedentroagliuraganisupereroicomeioeteseavraipauraallorastringimilemaniperchesiamoinvincibilivicinieovunqueandrosaraiconmesupereroisoloioetedueg

## Attacco

In [70]:
flag=False
for a in range(len(P)):
    for b in range(a+1,len(P)):
        if invertibile(P[[a,b],:]):
            flag=True
            break 

    if flag:
        break
print(a,b)
Past=P[[a,b],:]
Pastm1=inversa(Past)
print(Pastm1)
    

0 1


KeyboardInterrupt: 

In [73]:
round(sc.linalg.det(P[[a,b],:])%26)

13